In [1]:
import torch

import random
import numpy as np
from transformers import BertTokenizer


SEED = 321

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
from transformers import BertTokenizer

print('Loading Tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Loading Tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [3]:
print('Vocab Size:',len(tokenizer.vocab))

tokens = tokenizer.tokenize('This is a string to tokenize')

print('Tokenized list:', tokens) # Split sentence into tokens

indexes = tokenizer.convert_tokens_to_ids(tokens) 

print('Token IDS:', indexes) # print sentence mapped to tokens ids

Vocab Size: 30522
Tokenized list: ['this', 'is', 'a', 'string', 'to', 'token', '##ize']
Token IDS: [2023, 2003, 1037, 5164, 2000, 19204, 4697]


In [4]:
# CLS, SEP, PAD and UNK token
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

# INDEX
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

[CLS] [SEP] [PAD] [UNK]


In [5]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length) # 512

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

512


In [6]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut, # try to replace it with 'spacy'
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(split_ratio=0.8 ,random_state = random.seed(SEED))

print(f"Number of training examples: {len(train_data)}") # 20,000 examples
print(f"Number of validation examples: {len(valid_data)}") # 5,000 examples
print(f"Number of testing examples: {len(test_data)}") # 25,000 examples

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 36.8MB/s]
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [8]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[0])['text'])

print(tokens)

['one', 'of', 'the', 'most', 'un', '##her', '##ald', '##ed', 'great', 'works', 'of', 'animation', '.', 'though', 'it', 'makes', 'the', 'most', 'sophisticated', 'use', 'of', 'the', '"', 'cut', '-', 'out', '"', 'method', 'of', 'animation', '(', 'a', 'la', '"', 'south', 'park', '"', ')', ',', 'the', 'real', 'talent', 'behind', '"', 'twice', 'upon', 'a', 'time', '"', 'are', 'the', 'vocal', 'characterization', '##s', ',', 'with', 'lorenzo', 'music', "'", 's', '(', 'carlton', 'from', 'tv', "'", 's', '"', 'r', '##ho', '##da', '"', ')', 'woody', 'allen', '-', 'is', '##h', 'ralph', '-', 'the', '-', 'all', '-', 'purpose', '-', 'animal', 'being', 'the', 'center', '##piece', '.', 'the', '"', 'accidental', 'nightmare', '"', 'sequence', 'is', 'doubt', '##less', 'one', 'of', 'the', 'best', 'pieces', 'of', 'animation', 'ever', 'filmed', '.']


In [9]:
LABEL.build_vocab(train_data)
print('Number of Positive and Positive Sentiments:', LABEL.vocab.freqs)


Number of Positive and Positive Sentiments: Counter({'pos': 10009, 'neg': 9991})


In [10]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('The model is being trained on:', device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

The model is being trained on: cuda


/opt/conda/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Build the Model

In [11]:
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.nn.functional as F

# Define Bert model
bert = BertModel.from_pretrained('bert-base-uncased')


class BERTSentimentAnalysis(nn.Module):
    
    def __init__(self, bert, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)        
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
                
        with torch.no_grad():
            embedded = bert(text)[0]
        
        _, hidden = self.rnn(embedded)
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                    

        output = self.out(hidden)
        
        return output

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [12]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2

model = BERTSentimentAnalysis(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False
        
        
print(f'The model now has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters
The model has 2,759,169 trainable parameters


## Model Training


In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [15]:
# Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8

def accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float 
    acc = correct.sum() / len(correct)
    return acc


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    print("start time:", start_time)
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    print("-" * 30)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    print("end time:" ,end_time)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(epoch_mins, epoch_secs)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './New-Bert-model.pt')
    
    #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

start time: 1617205796.9786918


/opt/conda/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


------------------------------
end time: 1617206253.1845224
7 36
	Train Loss: 0.446 | Train Acc: 78.33%
	 Val. Loss: 0.297 |  Val. Acc: 88.05%
start time: 1617206254.4091597
------------------------------
end time: 1617206710.540974
7 36
	Train Loss: 0.257 | Train Acc: 89.53%
	 Val. Loss: 0.242 |  Val. Acc: 90.14%
start time: 1617206712.0723462
------------------------------
end time: 1617207168.330224
7 36
	Train Loss: 0.224 | Train Acc: 91.11%
	 Val. Loss: 0.249 |  Val. Acc: 90.04%
start time: 1617207168.3314047
------------------------------
end time: 1617207624.6007917
7 36
	Train Loss: 0.199 | Train Acc: 92.12%
	 Val. Loss: 0.217 |  Val. Acc: 91.56%
start time: 1617207626.12074
------------------------------
end time: 1617208082.4959943
7 36
	Train Loss: 0.173 | Train Acc: 93.38%
	 Val. Loss: 0.235 |  Val. Acc: 90.23%


We can see that Neural Network starts to overfit the data after the 5th EPOCH

In [18]:
# Uncomment this cell if you want to release memory of CUDA 

# !pip install GPUtil

# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# free_gpu_cache()  

We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [19]:
model.load_state_dict(torch.load('./New-Bert-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.196 | Test Acc: 92.25%


## Prediction Testing

In [36]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    if prediction.item() > 0.5:
        print('Movie review has a negative sentiment')
    else:
        print('Movie review has a positive sentiment')
    return prediction.item()

Reviews from critics are collected from Zack Snyder's cut of Justice League. Source - Rotten Tomatoes: https://www.rottentomatoes.com/m/zack_snyders_justice_league

In [37]:
predict_sentiment(model, tokenizer, "The new cut of Justice League is better than the theatre release of 'Justice League'. Unfortunately, that doesn’t make it a good movie")


Movie review has a negative sentiment


0.5217639207839966

In [41]:
predict_sentiment(model, tokenizer, '''Zack Snyder's Justice League shows Snyder's evolution from a director known for emphasizing style over substance to a man who understands why we love superhero myths, whether they live in the DC Universe or Marvel's.''')

Movie review has a positive sentiment


0.02805434726178646

In [48]:
predict_sentiment(model, tokenizer, '''Let's not forget that the Snyder Cut was in part willed into being by a toxic social media campaign, mob rule begetting a mystical monocultural object that is finally much bigger than its ostensible creator.''')


Movie review has a negative sentiment


0.826120913028717

In [54]:
predict_sentiment(model, tokenizer, '''It's a miracle this film was released 4 years later! ZS's Justice League will be remembered forever in cinema history - pleasing current DC fans and attracting new ones!''')


Movie review has a positive sentiment


0.047379642724990845

You are more than welcome to download trained BERT Model and test it yourself!